# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its output are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

In [1]:
from collections import Counter
from pyqrack import QrackSimulator

hidden_bits = 37
input_size = 8
num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        if ((hidden_bits >> i) & 1) > 0:
            sim.mcx([i], i + input_size)

All of the following statements of equality are true:

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# This measurement result is always the "hidden_bits" parameter of the oracle
results = Counter(sim.measure_shots(oracle_qubits, 10000))

for key in results.keys():
    print("hidden_string .", key, "= 0")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
hidden_string . 0 = 0
hidden_string . 2 = 0
hidden_string . 8 = 0
hidden_string . 10 = 0
hidden_string . 16 = 0
hidden_string . 18 = 0
hidden_string . 24 = 0
hidden_string . 26 = 0
hidden_string . 64 = 0
hidden_string . 66 = 0
hidden_string . 72 = 0
hidden_string . 74 = 0
hidden_string . 80 = 0
hidden_string . 82 = 0
hidden_string . 88 = 0
hidden_string . 90 = 0
hidden_string . 128 = 0
hidden_string . 130 = 0
hidden_string . 136 = 0
hidden_string . 138 = 0
hidden_string . 144 = 0
hidden_string . 146 = 0
hidden_string . 152 = 0
hidden_string . 154 = 0
hidden_string . 192 = 0
hidden_string . 194 = 0
hidden_string . 200 = 0
hidden_string . 202 = 0
hidden_string . 208 = 0
hidden_string . 210 = 0
hidden_string . 216 = 0
hidden_string . 218 = 0


...The ultimate point of the algorithm is that, by solving the above system of equations, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).